# 迭代器和生成器

⚠️本文有关迭代器的基础知识需要好好梳理以下

- 生成器是怎样节约内存的？
- 使用生成器的最佳时机是什么？
- 如何用itertools 来创建复杂的生成器工作流？
- 延迟估值何时有益，何时无益？

生成器的好处是：节省内存.

⚠️生成器的状态的数量很重要，如果保存的状态很少，比较适合生成器。反之，如果一个函数需要很多状态，却输出很少的数据，那么预先的列表会更好。

## 1. 生成器

在Python 中，我们通常使用for 循环：

```python
for i in range(10):
    print(i)
```

这里的range 函数就是一个生成器。

In [2]:
def my_range (start, stop, step=1):
    while start < stop:
        yield start
        start += step

In [4]:
for i in my_range(0,5):
    print(i)

0
1
2
3
4


`my_range` 函数每次执行到yield 的地方，发射出(emit)一个值，然后整个函数的状态保持在内存中，直到下次调用的时候继续执行。

### 迭代器

for 循环要求被循环的对象支持迭代，这就要求我们生产一个**迭代器**。我们可以使用Python内置的iter 函数。
- 数组，元组，字典，集合返回一个对象的内部元素(或键)的迭代器
- 对于复杂的对象，iter 函数返回对象的`__iter__` 方法

因此，下面两个方法等同。

In [14]:
class my_object:
    
    def __init__(self):
        self.lst = [1,2,3,4,5]
    
    def __iter__(self):
        return iter(self.lst)

my_obj = my_object()
my_obj.__iter__

<bound method my_object.__iter__ of <__main__.my_object object at 0x112244780>>

In [15]:
for i in my_obj:
    print(i)

1
2
3
4
5


In [17]:
lst_iter = iter(my_obj)

while True:
    try: 
        i = lst_iter.next()
        print(i)
    except StopIteration:
        break

AttributeError: 'list_iterator' object has no attribute 'next'

In [8]:
lst_iter

#### 使用生成器的好处

问题：找到一个列表中能被3整除的数的个数。


In [22]:
numbers = [x for x in range(100)]

In [25]:
result = len([n for n in numbers if n % 3 == 0])
result

34

上面这个方法使用了list comprehension，分配了内存，但是却没什么用（如果以后再不使用这个被3整除的数组）。

我们可以使用生成器，但是生成器没有length 属性，所以我们要换个思路。

In [28]:
result = sum((1 for n in numbers if n % 3 == 0))

In [29]:
result

34

这里，我们每次遇到一个能被3整除的数字就发射一个1，然后求和，就可以获得个数。两个代码的性能几乎一样，但是第二个版本所需要的内存远远小于第一个。

## 2. 无穷数列的迭代器

如果我们只需要保存有限个状态，并发射当前值，生成器可以被用来长生无穷数列，例如斐波纳西数列 ———— 保留两个状态，生成下一个数。

下面我们想打印出所有小于100的奇数的菲波纳西数列，我们有以下几个方法：

#### 方法1

In [47]:
def solution1():
    i, j = 0, 1
    count = 0
    while j < 100:
        
        if j % 2:
            print(j)
        
        i, j = j, i + j
        
solution1()

1
1
3
5
13
21
55
89


这种方法的问题在于，他一次做了很多事，隐藏了真实的计算逻辑，也不容易扩展。

比如，我们要求打印出"5000以内的xxxx数列中的奇数"，则需要要对代码有较大的改动。

#### 方法2

我们吸取方法1的教训，把逻辑拆开，分别为两个截断：
- 生成下一个菲波纳西数列的数（数据生成） --- 生成器
- 判断这个值是否小于100（数据转化/计算）--- 专注于计算代码

In [39]:
def fibonacci():
    i, j = 0, 1
    while True:
        yield j
        i, j = j, i + j

In [48]:
def solution2():
    for i in fibonacci():
        if i > 100:
            break
        if i % 2:
            print(i)

In [49]:
solution2()

1
1
3
5
13
21
55
89


我们可以看出，这个方法看上去很直观，易于开发者理解和调试。

在者，而且这个函数可以变得更加通用，可以不止对菲波纳西数列进行判断，也可以对任何一个数列的生成器进行判断。

#### 方法3

In [53]:
from itertools import islice

def solution3():
    cond = lambda x: x % 2 and x < 100  # 条件
    first_100 = islice(fibonacci(), 0, 10)  # 生成0 - 100 的斐波纳西数列 --- ⚠️THIS IS WRONG 因为我们不知道个数
    for x in first_100:
        if cond:
            print(x) 

In [54]:
solution3()

1
1
2
3
5
8
13
21
34
55


这个方法不是很好理解。

从上面3个方法我们可以看出，方法2是最好的方法。因为分工明确：生成器用于生成数据，其他函数用于处理(操作)数据。

## 3. 生成器的延迟估值

Benefits with a generator is by dealing only with the current values of interest.

在计算的任意时间点，我们都只能访问当前值，无法访问数列中的其他元素。single-pass / online

很多模块和函数可以配合生成器一起使用，例如，itertools。它提供了map, reduce, filter, zip 的生成器版本。
- islice: Allows slicing a potentially infinite generator
- chain: Chains together multiple generators
- takewhile: Adds a condition that will end a generator
- cycle: Makes a finite generator infinite by constantly repeating it

#### Example

对于一个时序数据，每秒钟一个点，一共20年。那么我们有20*365*365*24*60*60 = 630,720,000 个数据点(注意，我们按一年365天估算)。这将是一个很大的数据集，如果我们想做一些异常检测，我们无法一次读入内存。下面我们演示如何使用生成器来实现这个功能。

数据格式： `timestamp, value`

异常定义：对于每个数据点，比同一天均值超出三倍标准差之外的数字被定义为异常值(3-sigma 方法)。

⚠️这里有问题，代码跑不通，待第二版。

In [55]:
from random import normalvariate, rand
from itertools import count

def read_data(filename):
    """
    每次读取文件的一行
    """
    with open(filename) as fd:
        for line in fd:
            data = line.strip().split(',')
            yield map(int, data)

def read_fake_data(filename):
    for i in count():
        sigma = rand() * 10
        yield (i, normalvariate(0, sigma))

ImportError: cannot import name 'rand'

然后我们使用groupby 将同一天的timestamp 分在一组。

#### 输入：2个参数
- 一组数据
- 用于分组的关键字函数: 我们可以使用lambda 函数返回一个date 对象。

#### 返回：生成器

生成的每一个值都是一个元组，元组内包含该组的关键字和改组的所有元素的生成器。

#### 例如：
The only limitation is that groups will only be formed for data that is sequential.

如果我们给一个序列，AAABBAA，则返回三个groups:
- A: [A A A]
- B: [B B]
- A: [A A]

同一天的数据会有相同的date 对象，用于分组。

In [57]:
from datetime import date
from itertools import groupby

def day_grouper(iterable):
    
    key = lambda timestamp, value : date.fromtimestamp(timestamp)
    
    return groupby(iterable, key)

In [59]:
import math

def check_anomaly(day, day_data):
    # We find the mean, standard deviation, and maximum values for the day.
    # Using a single-pass mean/standard deviation algorithm allows us to only
    # read through the day's data once.
    n = 0
    mean = 0
    M2 = 0
    max_value = None
    for timestamp, value in day_data:
        n += 1
        delta = value - mean
        mean = mean + delta/n
        M2 += delta*(value - mean)
        max_value = max(max_value, value)
    variance = M2/(n - 1)
    standard_deviation = math.sqrt(variance)
    
    # Here is the actual check of whether that day's data is anomalous. If it
    # is, we return the value of the day; otherwise, we return false.
    if max_value > mean + 3 * standard_deviation:
        return day
    return False

## to-check

- linecache
- collections.deque